### Analyse search terms on the e-commerce web server


In [ ]:
# Install spark

In [3]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [ ]:
# Start session

In [2]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Saving and Loading a SparkML Model").getOrCreate()

23/06/04 02:22:18 WARN Utils: Your hostname, irteza resolves to a loopback address: 127.0.1.1; using 192.168.0.228 instead (on interface wlo1)
23/06/04 02:22:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/04 02:22:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [4]:
!curl https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv --output searchterms.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  227k  100  227k    0     0   127k      0  0:00:01  0:00:01 --:--:--  127k


In [ ]:
# Load the csv into a spark dataframe

In [19]:
sdf = spark.read.csv('searchterms.csv', header=True).cache()

In [21]:
print(f'Number of columns: {len(sdf.columns)}, Number of rows: {sdf.count()}') 


Number of columns: 4, Number of rows: 10000


In [22]:
sdf.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [18]:
sdf.schema['searchterm'].dataType

StringType()

In [23]:
sdf.createTempView('searchterms')

In [29]:
spark.sql("select count(*) from searchterms where searchterm='gaming laptop'").show()


+--------+
|count(1)|
+--------+
|     499|
+--------+



In [31]:
sdf.groupBy("searchterm").count().orderBy("count",ascending=False).show(5)

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



In [ ]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

In [34]:
!curl https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz --output mlmodel.tar.gz
!tar xzvf mlmodel.tar.gz 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1490  100  1490    0     0   1260      0  0:00:01  0:00:01 --:--:--  1260
sales_prediction.model/
sales_prediction.model/metadata/
sales_prediction.model/metadata/part-00000
sales_prediction.model/metadata/.part-00000.crc
sales_prediction.model/metadata/_SUCCESS
sales_prediction.model/metadata/._SUCCESS.crc
sales_prediction.model/data/
sales_prediction.model/data/part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet
sales_prediction.model/data/_SUCCESS
sales_prediction.model/data/.part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet.crc
sales_prediction.model/data/._SUCCESS.crc


In [37]:
from pyspark.ml.regression import LinearRegressionModel

model = LinearRegressionModel.load('sales_prediction.model')

In [42]:
def predict(year):
    assembler = VectorAssembler(inputCols=["year"],outputCol="features")
    data = [[year]]
    columns = ["year"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features')
    predictions = model.transform(__)
    predictions.select('prediction').show()

In [46]:
predict(2023)

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+

